<a href="https://colab.research.google.com/github/b-sagal/EEC-201-Final-Project/blob/main/Project_Phase_3_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Phase 3 of Project -- Using methodology from Phase 2, create a data preproecessing feature extraction pipeline for ML. Then begin exploratory analysis of different surpervised (v1) and unsupervised (v2) learning methods
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
from glob import glob
import scipy
import seaborn as sns

from itertools import cycle
from google.colab import drive
drive.mount('/content/drive')

sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

Mounted at /content/drive


In [ ]:
# Phase 2 -- Define different approaches to obtaining MFCC

# Approach 1 - Full Manual Calculation -- Unfinished

def manual_MFCC(x, sr, fft_length, window_length, overlap,num_mels ):

    length_of_sample = len(x)
    n = fft_length
    hoplength = n - overlap # Hop length is Window - Overlap. Part of a window with no overlap
    max_STFT_matrix_size = np.floor(length_of_sample / hoplength) # Calculate the integer number of full windows

    # STFT
    stft = librosa.stft(x, n_fft=n, win_length=window_length, window='hann', hop_length=hoplength)
    STFT_shape = stft.shape

    # Check if there is a "partial" vector and if there is delete it from the dataframe
    if STFT_shape[0] > max_STFT_matrix_size:

        stft = np.delete(stft, np.s_[-1:], axis=1)

    # Converter Complex FFT to Real FFT with Abs
    stft = abs(stft)

    # Create the Mel Filter Banks -- Will use default max frequency -> sr/2
    mel_fil_banks = librosa.filters.mel(sr=sr, n_fft=n, n_mels=num_mels)

    return

# Approach 2 - Fully automated using librosa's mfcc
def auto_MFCC(x, sr, fft_length, window_type, window_length, overlap, num_mels, num_mfcc, dct):

    #length_of_sample = len(x)
    fft_length
    hoplength = fft_length - overlap # Hop length is Window - Overlap. Part of a window with no overlap
    #max_STFT_matrix_size = np.floor(length_of_sample / hoplength) # Calculate the integer number of full windows

    mfcc = librosa.feature.mfcc(y=x, sr=sr, S=None, n_fft=fft_length, window=window_type, win_length=window_length, n_mels=num_mels, n_mfcc=num_mfcc, dct_type=dct, hop_length=hoplength)

    return mfcc


In [ ]:
# Phase 2 -- Get a list of Audio Files

audio_files = glob('/content/drive/MyDrive/Colab Notebooks/training_data_201' + '/*.wav')
y, sr = librosa.load(audio_files[10])



11


In [ ]:
# Phase 2 -- Looping through the audiofiles, calculating the MFCC, and placing in a list
'''
unlabled_speaker_mfccs = []
labled_speaker_mfccs = []

# Define MFCC parameters
n_fft = 256
window_type = 'hann'
window_length = 200
overlap = 100
n_mels = 3
n_mfcc = 10
dct = 2

# Loop through audio_files list and perform MFCC
i = 0
for audio_file in audio_files:

    y, sr = librosa.load(audio_file)

    mfcc = auto_MFCC(y, sr, n_fft, window_type, window_length, overlap, n_mels, n_mfcc, dct)
    #print(mfcc.shape)
    # Append to unlabled list
    unlabled_speaker_mfccs.append(mfcc)


    # Create label vector append to mfcc and add to labeled list -- For supervised learning step
    shape_mfcc = mfcc.shape
    #print(shape_mfcc)
    class_label = np.full(shape_mfcc[1],i)
    #print(class_label)
    labled_mfcc = np.vstack([mfcc, class_label])
    #print(labled_mfcc)
    labled_speaker_mfccs.append(labled_mfcc)
    i+=1
'''

AttributeError: 'list' object has no attribute 'shape'

In [49]:
# Phase 3 -- ML Algorithm Exploration

# Define the data preprocessing pipeline for supervised learning and unsupervised learning

def mfcc_preprocessing_pipeline(files, n_fft, window_type, window_length, overlap, n_mels, n_mfcc, dct, mfcc_auto_mode):

    # Empty lists for data
    unlabeled_speaker_mfccs = []
    labeled_speaker_mfccs = []
    speaker_mfccs_dict = {}

    # Loop through audio_files list and perform MFCC
    i = 0
    for file in files:

        y, sr = librosa.load(file)

        if mfcc_auto_mode:

            mfcc = auto_MFCC(y, sr, n_fft, window_type, window_length, overlap, n_mels, n_mfcc, dct)

        else:

            mfcc = manual_MFCC(y, sr, n_fft, window_type, window_length, overlap, n_mels, n_mfcc, dct)


        # Drop the first mfcc -> DC mfcc typically unused
        #mfcc = np.delete(mfcc_full, 0, 0)

        # Create dictionary for each speakers mfccs
        speaker_mfccs_dict[i] = mfcc


        # Append to unlabled list
        unlabeled_speaker_mfccs.append(mfcc)

        # Create label vector append to mfcc and add to labeled list -- For supervised learning step
        shape_mfcc = mfcc.shape
        class_label = np.full(shape_mfcc[1],i)
        labeled_mfcc = np.vstack([mfcc, class_label])
        labeled_speaker_mfccs.append(labeled_mfcc)
        print(i)
        i+=1

    # Horizontally stack the list of numpy arrays to consolidate the data
    unlabeled_data = np.hstack(unlabled_speaker_mfccs)
    labeled_data = np.hstack(labled_speaker_mfccs)

    # Transpose to place features and class labels along the columns
    unlabeled_data = np.transpose(unlabeled_data)
    labeled_data = np.transpose(labeled_data)


    return unlabeled_data, labeled_data, speaker_mfccs_dict


In [50]:
# Phase 3v1 - Test the preprocessing pipeline. Begin the SL exploration

# Define MFCC parameters
n_fft = 400
window_type = 'hann'
window_length = 256
overlap = 100
n_mels = 100
n_mfcc = 10
dct = 2

mfcc_unlabeled, mfcc_labeled, mfcc_dict = mfcc_preprocessing_pipeline(audio_files, n_fft, window_type, window_length, overlap, n_mels, n_mfcc, dct, mfcc_auto_mode=True)
print(mfcc_labeled.shape)
print(mfcc_labeled)


0
1
2
3
4
5
6
7
8
9
10
(2158, 4)
[[-1.26505905e+02  4.70990038e+00 -2.34689713e-02  0.00000000e+00]
 [-1.24006111e+02  1.92233586e+00  9.91172969e-01  0.00000000e+00]
 [-1.21682938e+02  4.38168621e+00  1.20557070e-01  0.00000000e+00]
 ...
 [-1.24244797e+02  7.62604618e+00  1.61987758e+00  1.00000000e+01]
 [-1.19957909e+02  1.10061588e+01  4.62399578e+00  1.00000000e+01]
 [-1.17528511e+02  1.18090858e+01  3.36557865e+00  1.00000000e+01]]


In [ ]:
6#LBG Averaging
'''
#takes our data and averages the Nth users first mffcc coefficients to N coefficients and places them into the codebook vector, this is done for all N Users
drows, dcolumns = mfcc_labeled.shape

## our data in is a total of some rows, each column is a mfcc and the last element in each column is our label;
codebook = []
codevector = []
user_label = 0;
for x in range(0, drows-1):
  if mfcc_labeled[x][dcolumns-1] == user_label:
    codevector.append(mfcc_labeled[x])
  if mfcc_labeled[x][dcolumns-1] != user_label:
    if mfcc_labeled[x][dcolumns-1] == 11:
      print("yes")
    #now after all the data for the first user is collected we average each first mfcc coefficients -> N mffcc coefficients and store the averages in the codebook
    averages = [sum(filter(None, column)) / len(column) for column in zip(*codevector)]
    codebook.append(averages)
    #print(user_label)
    user_label = user_label + 1
    codevector = []
    #added so we dont miss the first frame mfccc coeffficients for the next user
    codevector.append(mfcc_labeled[x])
    #print(mfcc_labeled[x][dcolumns - 1])

#last element of each list is the label of the user, preceding numbers are the averages
print(codebook)
'''




[[-83.40592890900452, 14.793168043161367, 1.8419130154244312, 0.0], [-85.44687182698028, 11.374151288278846, -0.04860933292780491, 1.0], [-83.89158126286098, 17.98018472251438, 3.6698928131234076, 2.0], [-77.6395966613177, 15.15861477550951, -0.03880268914028279, 3.0], [-81.18979481288365, 11.990084417519116, 0.9809783519733519, 4.0], [-79.33642151019325, 14.755037212847201, 0.11094126151017616, 5.0], [-83.03760114922581, 16.25029438446803, 2.9921091579529175, 6.0], [-108.61346268820596, 28.71447469637944, 11.247175125808983, 7.0], [-112.15199385478998, 31.622371708511547, 11.294439369705833, 8.0], [-104.99706351451384, 27.60054977531107, 10.394788298851404, 9.0]]


In [ ]:
#LBG Algorithm
'''
# Define LBG Function - First Attempt
def lbg_algorithm(data, codebook_size, epsilon=0.01, max_iterations=100):
    # Initialize codebook with random vectors from data
    codebook = np.random.randn(codebook_size, len(data[0]))

    # Normalize codebook
    codebook = codebook / np.linalg.norm(codebook, axis=1)[:, np.newaxis]

    distortion = 0

    for _ in range(max_iterations):
        # Compute Euclidean distances between data points and codebook vectors
        distances = np.sqrt(((data[:, np.newaxis] - codebook)**2).sum(axis=2))

        # Assign each data point to the nearest codebook vector
        assignments = np.argmin(distances, axis=1)

        # Update codebook vectors
        new_codebook = np.zeros_like(codebook)
        for i in range(codebook_size):
            members = data[assignments == i]
            if len(members) > 0:
                new_codebook[i] = np.mean(members, axis=0)
            else:
                # If no data points assigned to a centroid, don't update it
                new_codebook[i] = codebook[i]

        # Calculate distortion
        new_distances = np.sqrt(((data[:, np.newaxis] - new_codebook)**2).sum(axis=2))
        new_assignments = np.argmin(new_distances, axis=1)
        new_distortion = np.mean(np.sqrt(((data - new_codebook[new_assignments])**2).sum(axis=1)))

        # Check convergence
        if abs(distortion - new_distortion) < epsilon:
            break

        codebook = new_codebook
        distortion = new_distortion

    return codebook

# Apply LBG algorithm
codebook_size = 55  # Hyperparameter for model tuning
codebook = lbg_algorithm(mfcc_unlabeled, codebook_size)
print(codebook.shape)
'''

(55, 3)


In [51]:
# LBG Algorithm - Second Attempt: Still need to build out classification feature of model

# Building core functions of algorithm

# Step 1

def initialize_codebook(mfcc_data):

    #Centroid list
    codebook = []

    # Initialize empty list for distances of the codebook
    #distances = []

    # Calculate the inital centroid -> mean vector of the entire mfcc for the speaker
    centroid = (mfcc_data.mean(1))
    codebook.append(centroid)
    '''
    Mean probably returns a numpy array anyway
    codebook_list.append(initial_centroid)
    codebook = np.array([codebook_list]).T
    '''

    # Calculate the inital distance -> euclidean distance between the mfcc vectors and initial centroid vector
    eu_distance = np.linalg.norm((mfcc_data.T - centroid.T).T, axis=0) # initial distance
    initial_avg_distance = eu_distance.T.mean()

    return codebook, initial_avg_distance


# Step 2

def double_codebook(codebook, epsilon):

    new_codebook = []

    # Split the centroids into two new centroids based on epsilon, add them to the list and remove the old centroid
    for centroid in codebook:

        new_centroid_1 = centroid - epsilon

        new_centroid_2 = centroid + epsilon

        new_codebook.extend([new_centroid_1, new_centroid_2])

    return new_codebook


# Step 3

def nearest_neighbor_search_classifier(input_vectors, codebook_ref):

    vector_classifier_dict = {}
    dimensions = input_vectors.shape
    rows, columns = dimensions

    for i in range(columns):

        distortions_list = []
        vector = input_vectors[:,i]

        for cluster in codebook_ref:

                eu_distance = np.linalg.norm(vector - cluster, axis=0)
                distortions_list.append(eu_distance)

        distortions_array = np.array(distortions_list)
        minimum_distortion = np.argmin(distortions_array)
        vector_classifier_dict[i] = minimum_distortion

    return vector_classifier_dict


# Step 4

def sequester_matrix_of_centroid_vectors(input_vectors, codebook_ref, vector_classifier_dict):

    codebook_dict = {}
    codebook_matrix_dict = {}
    dimensions = input_vectors.shape
    rows, columns = dimensions

    for i, clusters in enumerate(codebook_ref):

        codebook_dict[i] = []


    for i in range(columns):

        vector = input_vectors[:,i]

        cluster_index = vector_classifier_dict[i]

        codebook_dict[cluster_index].append(vector)


    for i, cluster in codebook_dict.items():

        codebook_matrix_dict[i] = np.array(cluster).T
        print(codebook_matrix_dict[i].shape)


    return codebook_matrix_dict


# Step 5

def new_centroid_locations (codebook_matrix_dict):

    new_centroids = []

    i = 0
    for classified_matrix in codebook_matrix_dict.values():

        centroid = np.array(classified_matrix).mean(1)

        new_centroids.append(centroid)

    return new_centroids



# Step 6

def total_avg_distortion_vector_to_clusters_1(vectors, codebook_ref, classification_dict):


    distances = []
    dimensions = vectors.shape
    rows, columns = dimensions

    for i in range(columns):

        vector = vectors[:,i]

        eu_distance = np.linalg.norm(vector - codebook_ref[classification_dict[i]], axis=0)

        distances.append(eu_distance)

    avg_distance = np.array(distances).T.mean()

    return avg_distance




# Execution of Algorithm
def lbg_algorithm_2(data, epsilon, max_iterations, codebook_size):

    # Step 1 - Initialize Codebook
    codebook, previous_distortion = initialize_codebook(data)

    # Outer Loop (Iteration 2)
    while len(codebook) < codebook_size:

        # Step 2 - Double Codebook Size
        codebook = double_codebook(codebook, epsilon)

        # Inner Loop (Iteration 1)
        distortion_break_condition = 1
        num_iter = 0
        while distortion_break_condition > epsilon: # or num_iter < max_iterations: <- Don't think this is needed since it seems to converge quickly. Plus this implementation is wrong

            if num_iter != 0:
                previous_distortion = new_distortion

            # Step 3 - Nearest Neighbor Search and Classify
            classified_data_dict = nearest_neighbor_search_classifier(data, codebook)

            # Step 4 - Wrangle data into classified matrices
            codebook_matrix_dict = sequester_matrix_of_centroid_vectors(data, codebook, classified_data_dict)

            # Step 5 - Find New Centroids
            codebook = new_centroid_locations (codebook_matrix_dict)

            # Step 6 - Calculate new centroids
            new_distortion = total_avg_distortion_vector_to_clusters_1(data, codebook, classified_data_dict)

            # Step 7 - Check if distortion is less than epsilon
            distortion_break_condition = (previous_distortion - new_distortion) / new_distortion

            num_iter += 1

    return codebook


In [52]:
# Testing LBG - Test speaker 1 - Codebook Size > 16 Seems to cause issues
#print(mfcc_dict[1].shape)
#codebook = lbg_algorithm_2(mfcc_dict[1], epsilon=0.01, max_iterations=100, codebook_size = 20)

In [53]:
# Loop through data and create full codebook

codebook = {}

for speaker, data in mfcc_dict.items():

    desired_codebook_size = 50 # Initialize to large number
    success_flag = False # Initialize success flag

    while success_flag == False:

        try:

            print(f'Speaker: {speaker}')
            codebook[speaker] = lbg_algorithm_2(mfcc_dict[speaker], epsilon=0.001, max_iterations=1000, codebook_size = desired_codebook_size)

            success_flag = True

        except:

            desired_codebook_size = desired_codebook_size - 2

            print(f"Desired Codebook Size: {desired_codebook_size}")

            succes_flag = False

    print("End of Loop")

Streaming output truncated to the last 5000 lines.
(10, 20)
(10, 7)
(10, 6)
(10, 7)
(10, 5)
(10, 17)
(10, 13)
(10, 48)
(10, 19)
(10, 7)
(10, 6)
(10, 7)
(10, 5)
(10, 17)
(10, 13)
(10, 25)
(10, 23)
(10, 9)
(10, 10)
(10, 3)
(10, 4)
(10, 3)
(10, 3)
(10, 4)
(10, 3)
(10, 3)
(10, 2)
(10, 8)
(10, 9)
(10, 7)
(10, 6)
(10, 26)
(10, 22)
(10, 7)
(10, 12)
(10, 3)
(10, 4)
(10, 2)
(10, 4)
(10, 3)
(10, 5)
(10, 3)
(10, 1)
(10, 7)
(10, 10)
(10, 6)
(10, 7)
(10, 25)
(10, 21)
(10, 6)
(10, 12)
(10, 7)
(10, 3)
(10, 3)
(10, 3)
(10, 2)
(10, 6)
(10, 3)
(10, 1)
(10, 7)
(10, 10)
(10, 7)
(10, 6)
(10, 26)
(10, 20)
(10, 6)
(10, 12)
(10, 7)
(10, 4)
(10, 2)
(10, 3)
(10, 2)
(10, 6)
(10, 3)
(10, 1)
(10, 7)
(10, 10)
(10, 7)
(10, 6)
(10, 24)
(10, 22)
(10, 6)
(10, 12)
(10, 7)
(10, 4)
(10, 2)
(10, 3)
(10, 2)
(10, 6)
(10, 3)
(10, 1)
(10, 7)
(10, 10)
(10, 7)
(10, 6)
(10, 24)
(10, 22)
(10, 6)
(10, 12)
(10, 7)
(10, 4)
(10, 2)
(10, 3)
(10, 2)
(10, 6)
(10, 3)
(10, 1)
(10, 7)
(10, 10)
(10, 7)
(10, 6)
(10, 21)
(10, 25)
(10, 6)
(10, 

In [57]:
# Prediction Method

# Load in test data
test_audio_files = glob('/content/drive/MyDrive/Colab Notebooks/Test_Data_201/*.wav')
y, sr = librosa.load(test_audio_files[0])
#print(len(test_audio_files))

# Define MFCC parameters
n_fft = 400
window_type = 'hann'
window_length = 256
overlap = 100
n_mels = 100
n_mfcc = 10
dct = 2

# Preprocess audio data
mfcc_unlabeled, mfcc_labeled, mfcc_dict = mfcc_preprocessing_pipeline(test_audio_files, n_fft, window_type, window_length, overlap, n_mels, n_mfcc, dct, mfcc_auto_mode=True)


# Loop through audio data and predict

speaker_classification = {}

for speaker, data in mfcc_dict.items():

    speaker_classification_distortions = []

    for speaker_id, codewords in codebook.items():

        vector_classification_dict = nearest_neighbor_search_classifier(data, codewords)
        distortion = total_avg_distortion_vector_to_clusters_1(data, codewords, vector_classification_dict)

        speaker_classification_distortions.append(distortion)

    distortions_array = np.array(speaker_classification_distortions)
    minimum_distortion = np.argmin(distortions_array)

    speaker_classification[speaker] = minimum_distortion + 1

print(speaker_classification)

0
1
2
3
4
5
6


<ipython-input-49-be704a2195ee>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


7
{0: 11, 1: 1, 2: 6, 3: 3, 4: 4, 5: 6, 6: 6, 7: 7}
